In [ ]:
# Problem Statement:

# You're tasked with identifying "hidden gem" startups from the attached CSV— companies that are not heavily funded, but are showing strong performance metrics. Select companies that meet the following conditions:

# -Have an ARR over $100M
# -Have a Valuation under $500M
# -Have a G2 Rating of 4.0 or above
# -Were founded in or after 2015

# You must:
# 1)Group the results by Industry
# 2)For each industry, calculate:
# 3)The number of such companies
# 4)The average ARR
# 5)The average Valuation
# 6)Sort the industries by average ARR descending

# Display only industries that have at least 2 companies matching the above criteria.
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import col, regexp_replace, when, trim, upper

spark= SparkSession.builder.appName("pyspark_test").getOrCreate()
df = spark.read.csv("/content/top_100_saas_companies_2025.csv",header=True , inferSchema=True)

df1= df.dropna(subset=['ARR','Valuation','G2 Rating'])
df1 = df1.withColumn('G2 Rating', col('G2 Rating').cast("double"))
cols_to_convert = ["ARR", "Valuation", "Total Funding"]


def convert_unit(col_name: str):

    cleaned = upper(trim(regexp_replace(col(col_name), "[$,]", "")))

    return (
        when(cleaned.endswith("T"), regexp_replace(cleaned, "T", "").cast("double") * 1e12)
        .when(cleaned.endswith("B"), regexp_replace(cleaned, "B", "").cast("double") * 1e9)
        .when(cleaned.endswith("M"), regexp_replace(cleaned, "M", "").cast("double") * 1e6)
        .when(cleaned.endswith("K"), regexp_replace(cleaned, "K", "").cast("double") * 1e3)
        .when(cleaned.rlike("^[0-9.]+$"), cleaned.cast("double"))
        .otherwise(None)
    )
for c in cols_to_convert:
    df1 = df1.withColumn(c, convert_unit(c))

df2 = df1.filter((col('ARR')>1E8) | (col('Valuation')<5E8 )|(col('G2 Rating')>=4) | (col('Founded Year')>=2015))
df3 = df2.groupBy('Industry').agg(count("*").alias("Company Count"),avg('ARR').alias('avg_arr'),avg('Valuation').alias('avg_valuation')).orderBy('avg_arr',ascending=False)
df3.show()

+--------------------+-------------+---------+-------------+
|            Industry|Company Count|  avg_arr|avg_valuation|
+--------------------+-------------+---------+-------------+
| Enterprise Software|            2|1.5125E11|    1.6075E12|
|Database & Enterp...|            1|  5.29E10|       3.5E11|
|                 CRM|            1|  3.79E10|     2.278E11|
|   Creative Software|            1|  1.94E10|       2.4E11|
|            Payments|            2| 1.685E10|       7.5E10|
|  Financial Software|            1|  1.44E10|       1.8E11|
|IT Service Manage...|            1|    8.9E9|      1.47E11|
|        HR & Finance|            1|    7.3E9|       6.5E10|
|          E-commerce|            1|    7.1E9|       9.5E10|
|       Cybersecurity|            2|    5.3E9|      8.25E10|
|Video Communications|            1|    4.5E9|       8.5E10|
|     Restaurant Tech|            1|    3.9E9|       2.0E10|
|Collaboration Sof...|            1|    3.5E9|       5.5E10|
|      Communications|  